# `StencilGen` tutorial

This tutorial aims to give an overview of the DevitoBoundary StencilGen object, its purpose, and how to use it.

# Immersed boundaries
Partial differential equation (PDE) solvers based on finite-difference (FD) methods have found use in numerous scientific and engineering applications including seismic modelling and computational fluid dynamics. They benefit from comparitive simplicity, alongside relatively low memory and computational requirements versus other common solver types, as a product of their structured computational grids. However, an implication of such regular grids is the difficulty of solving equations in complex domains, as unlike structured meshes, the grid cannot be easily deformed to match the desired geometry. As such, much attention has been given to accurately solving PDEs in complex geometries without the need to resort to unstructured meshes.

One crude means of doing this is simply by applying boundary conditions at the grid nodes closest to the edge of the domain. However, this is not ideal, as the boundary becomes "staircased": a smooth, sloping boundary becomes a rough cascade of alternating horizontal and vertical steps. This is problematic is many situations, for example, introducing significant artificial scattering from the edge of the domain in wave propogation applications.

More sophisticated approaches include multiscale methods, where increasingly fine subgrids are introduced in the region of the boundary, such that the magnitude of any steps is limited, and curvilinear grids, where a coordinate transform is applied to deform the grid such that the nodes conform with the geometry of the computational domain. However, both of these methods introduce complexity through additional subgrids in the former case, and geometric transformations in the latter.

An alternative approach is the use of an immersed boundary. Immersed boundary methods enable the imposition of boundary conditions at points which do not coincide with grid nodes. As such, the boundary surface can be defined independently of the grid, allowing for complex domains to be embedded within a (hyper)rectangular computational grid.

In [1]:
from devitoboundary.stencils.stencil_utils import generic_function
from devitoboundary.symbolics.symbols import x_b

We will begin by defining some boundary conditions that we want to impose. In Devitoboundary, these are specified using `x_b` and `generic_function`

In [2]:
help(generic_function)

Help on function generic_function in module devitoboundary.stencils.stencil_utils:

generic_function(val, deriv=0)
    Returns specified derivative of a polynomial series. To be used in the place
    of functions for specification of boundary conditions.
    
    Parameters
    ----------
    val : Sympy symbol
        The variable of the function: x_b should be used.
    deriv : int
        The order of the derivative. Default is zero.



For the purposes of boundary condition definition, it is not necessary to know the physical meaning of the function on which conditions are to be imposed. It may be that there are several fields on which the same boundary conditions are to be applied. As such, Devitoboundary encapsulates the function to which boundary conditions are to be applied using `generic_function`.

In [3]:
# Second derivative of some function
generic_function(x_b, 2)

Sum(n*x_b**n*(n - 1)*a[n], (n, 0, n_max))/x_b**2

As we can see, calling `generic_fuction(x_b, 2)` gives the second derivative of a polynomial series of order $n_{max}$. The reasoning behind the use of a polynomial series to represent the function will become clear later in this notebook.

`x_b` is simply a SymPy symbol used throughout Devitoboundary to refer to some arbitrary position on the boundary. As such, we can specify the boundary condition $\frac{\partial^{2} f}{\partial x^{2}}(x_{b}) = 0$ as follows:

In [4]:
from devito import Eq

Eq(generic_function(x_b, 2), 0)

Eq(Sum(n*x_b**n*(n - 1)*a[n], (n, 0, n_max))/x_b**2, 0)

We can define several boundary conditions in a list to pass to the `StencilGen` object:

In [5]:
# Define the spatial discretization order of our scheme
space_order = 4

# Zero even derivatives on the boundary
bcs = [Eq(generic_function(x_b, 2*i), 0) for i in range(int(space_order/2))]

bcs

[Eq(Sum(x_b**n*a[n], (n, 0, n_max)), 0),
 Eq(Sum(n*x_b**n*(n - 1)*a[n], (n, 0, n_max))/x_b**2, 0)]

# Calculating extrapolation polynomials
As has been pointed out, the "generic function" is approximated via some extrapolation polynomial. We will now consider the calculation of the coefficients of this polynomial. Each coefficient will be a function of boundary position, some arbitrary positions, and respective function values at these points. Appropriate substitution of these values will allow the polynomial to be evaluated at exterior stencil points.

This extrapolation of interior function values onto exterior points is the core of all immersed boundary implementations. You may have noted that the extrapolation used by Devitoboundary is in 1D. This is a common approach to take to immersed boundary methods, as it becomes increasingly difficult to determine extrapolation coefficients for higher order schemes. The orientation of this extrapolation within the higher-dimensional space is dependent upon the implementation. `StencilGen` assumes that extrapolations  are axially aligned for each dimension.

By substituting the extrapolated values for exterior stencil points into the stencil expression, a modified set of coefficients are obtained.

In [6]:
from devitoboundary import StencilGen

# The location of Devitoboundary's main stencil cache
cache = "../../devitoboundary/stencil_cache.dat"
sten_gen = StencilGen(space_order, bcs, stencil_file=cache)

To create a `StencilGen` object, we must specify the order of our spatial discretization and a list of boundary conditions. Additionally, a stencil file where calculated stencil sets are cached can be specified, to avoid needing to repeatedly solve for extrapolation coefficients.
# Stencil variants
To ensure that extrapolations do not become excessively rough, stencil points which lying close to the boundary are excluded. Half a grid increment was proposed as the threshold at which points are considered to be "close" by Wim Mulder in his 2017 paper (https://doi.org/10.1093/gji/ggx178). This is a convention that Devitoboundary also uses.

Following from this design choice, it becomes apparent that for each combination of discretization order, derivative order, and evaluation offset, there is a characteristic set of modified stencils. The degree of truncation on either side can be categorised as a "variant" representing a unique combination of exterior points and points unavailable for the extrapolation. Defining $\eta$ as distance from the current position to the boundary along the axis, measured in grid increments, then variants for a side are as follows:

![Title](variants_diagram.png)

with zero being no truncation. Each variant corresponds with an $\eta$ interval of a half, with larger variant numbers corresponding with increased truncation. There are $(order+1)$ variants per side, resulting in $(order+1)^{2}$ unique stencil modifications.